# Homework 3 - Interactive Viz

In [ ]:
import folium
import requests as rq
from bs4 import BeautifulSoup as bfs
import numpy as np
import pandas as pd
import collections
import os
%matplotlib inline
import matplotlib.pyplot as plt
from dateutil import relativedelta
import scipy.stats as stats
import math
import json
from pprint import pprint
from ipywidgets import FloatProgress
from IPython.display import display
from geopy.geocoders import GeoNames, Nominatim, GoogleV3
import locale

topojson_cantons = r'ch-cantons.topojson.json'

## Recommendations (Test Map)

"I also recommend you to use an intermediate viz step for debugging purposes, showing all the universties as markers in your map"

First let's begin to make an intermediate vizualisation : 
    - The Approved Amount is randomly generated 
    - We get the different name of canton in topojson_cantons file

In [ ]:
with open(topojson_cantons) as data_file:    
    ch_cantons_json = json.load(data_file)

# We loop on the json file in order to get the initials of each canton.
# Moreover we associate a random number to each cantons.
cantons_id = [canton['id'] for canton in ch_cantons_json['objects']['cantons']['geometries']]
data_test = pd.DataFrame({
        'canton': cantons_id,
        'value': np.random.randint(0, 100, len(cantons_id))
    })
data_test.head()

In [ ]:
# We display the map with the dataframe built previously.
map_test = folium.Map(location=[46.8, 8.1], zoom_start=8)
map_test.choropleth(
    geo_path=topojson_cantons,
    topojson='objects.cantons',
    data=data_test,
    columns=['canton', 'value'],
    key_on='feature.id',
    fill_color='PuBuGn',
    fill_opacity=0.5,
    line_opacity=0.7,
    threshold_scale=list(np.linspace(data_test['value'].min(), data_test['value'].max(), 6)),
    reset=True
)
map_test

## SNSF data

### Grant CSV file

After downloading the P3_grantExport, we open it with a data parser in order to manipulate date more easily.

In [ ]:
snsf_data = pd.read_csv('Data/P3_GrantExport.csv', sep=';', parse_dates=['Start Date', 'End Date'])
snsf_data.head()

In [ ]:
snsf_data.dtypes

### Data processing

Before putting the colums 'Project Number' as index of the dataframe we need to check if it's unique.

In [ ]:
snsf_data.columns = snsf_data.columns.str.replace(snsf_data.columns[0], 'Project Number')
snsf_data['Project Number'].is_unique

In [ ]:
snsf_data.set_index('Project Number', inplace=True)
snsf_data.head()

As we can see with the dtypes method the values in 'Approved Amount' are objects.
We need cast the differents values to numeric in order to be able to compute the sum easily. <br/>


Here the can notice that 10910 value cannot be cast to numeric for two reason find in documentation :

    - "This amount is not indicated in the case of mobility fellowships since it depends on administrative factors,
    typically the destination, cost of living, family allowances (if applicable) and exchange rate differences."
    
    - "Also in the case of NCCRs this amount is not available in P3"
     
 
>*After trying to get the result from NCCR we can conclude: <br/>
From the http://www.snf.ch/SiteCollectionDocuments/nfs/nccr_guide_2016.pdf we can notice the cantons who reveive
the nccr financial help are the cantons who reveive the most money from the P3 file. The result should not change
a lot if we take in account the cnnr money.*

In [ ]:
snsf_data['Approved Amount'] = pd.to_numeric(snsf_data['Approved Amount'], errors='coerce')
snsf_data['Approved Amount'].value_counts(dropna=False).head()

*"This is the institution where the project will largely be carried out according to the application. Pick list. This field is only filled if the research is carried out at a Swiss institution, otherwise the field remains blank. In the case of mobility fellowships, it is generally left empty." <br/> *

> As our study should be on swiss data, we decide to remove all null value.

In [ ]:
# We keep only the university not null, because those university are swiss.
snsf_data_swiss = snsf_data[snsf_data['University'].notnull()].copy()
snsf_data_swiss.head()

In [ ]:
# We check if we don't have Nan value anymore.
snsf_data_swiss['University'].value_counts(dropna=False).head()

let's see if the mobility fellowship is  really important in the data set.

In [ ]:
snsf_data_swiss[snsf_data_swiss['Funding Instrument'].str.contains("fellowships")]['Funding Instrument'].value_counts(dropna=False)

In [ ]:
snsf_data_swiss[snsf_data_swiss['Funding Instrument'].str.contains("Mobility")]['Funding Instrument'].value_counts(dropna=False)

## Find location with university names

In [ ]:
# We get all the swiss university.
universities_list = snsf_data['University'].dropna().unique()
universities_list

In [ ]:
# Use a given Geocoders from geopy to locate a given university in the Switzerland country
# geolocator: the Geocoders instance
# file_path: file where result data will be keep, if the file already exist no now request will be done
# universities: list of all universities to find
def find_location(geolocator, file_path, universities=universities_list):

    if os.path.exists(file_path):
        serie = pd.read_pickle(file_path)
    else:
        match_location = {}
        progress_bar = FloatProgress(min=0, max=len(universities))
        display(progress_bar)

        for university in universities:
            match_location[university] = geolocator.geocode(university + ' CH')
            progress_bar.value += 1

        serie = pd.Series(match_location)
        serie.to_pickle(file_path)

    return serie

### Token access

In [ ]:
env_file = r'env.json'

with open('env.json') as file:    
    env = json.load(file)

### Fetch from GeoNames

In [ ]:
geo_geonames = GeoNames(username=env['GeoNames-Username'])
data_location_geonames = find_location(geo_geonames, 'Data/geonmaes_locations.pickle')
data_location_geonames.value_counts(dropna=False)

### Fetch from OpenStreetMap

In [ ]:
geo_nominatim = Nominatim()
data_location_nominatim = find_location(geo_nominatim, 'Data/nominatim_locations.pickle')
data_location_nominatim.value_counts(dropna=False)

### Fetch from  Google Maps

In [ ]:
geo_google = GoogleV3(api_key=env['GoogleMaps-Key'])
data_location_google = find_location(geo_google, 'Data/google_locations.pickle')
data_location_google.value_counts(dropna=False)

We have noticed that even by using three differents API the results are insufficient. We have decided to complete the missing data manually.

To do so, we merge all results from API to make a single CSV file with all universities, canton and location. This file is temporary.

In [ ]:
universities_cantons_geo = pd.DataFrame(snsf_data['University'].dropna().unique(), columns=['University'])  # We retrieve all universities

universities_cantons_geo.head()

# We convert series created by find_location function into DataFrame containing two columns: University and Location
df_location_geonames = pd.DataFrame({'Location':data_location_geonames}).dropna()
df_location_geonames.index.name = 'University'
df_location_geonames.reset_index(inplace=True)
df_location_nominatim = pd.DataFrame({'Location':data_location_nominatim}).dropna()
df_location_nominatim.index.name = 'University'
df_location_nominatim.reset_index(inplace=True)
df_location_google = pd.DataFrame({'Location':data_location_google}).dropna()
df_location_google.index.name = 'University'
df_location_google.reset_index(inplace=True)

def append_latitude(row, type):
    '''
    This functions check if a location has been found by a Geolocator for university passed as parameter.
    If so, it returns latitude (respectively longitude).
    
    row: university (containing university name and location, if any)
    type: type of coordinate we want to retrieve if a location is set
    '''
    
    results = df_location_geonames[df_location_geonames['University'] == row['University']]['Location']
    if results.empty:
        results = df_location_nominatim[df_location_nominatim['University'] == row['University']]['Location']
    if results.empty:
        results = df_location_google[df_location_google['University'] == row['University']]['Location']

    if not results.empty:
        # First index filter: accessing first result of series 'results'
        # Second index filter: accessing the coordinates
        # Third index filter: accessing the latitude (respectively longitude)
        return results.values[0][1][0 if type == 'Latitude' else 1]
    else:
        return ''

universities_cantons_geo['Latitude'] = universities_cantons_geo.apply(lambda row: append_latitude(row, 'Latitude'), axis=1)
universities_cantons_geo['Longitude'] = universities_cantons_geo.apply(lambda row: append_latitude(row, 'Longitude'), axis=1)

# We save our results in a CSV file
universities_cantons_geo.set_index('University').to_csv('Data/universities_cantons_geo_tmp.csv')

In [ ]:
!head Data/universities_cantons_geo_tmp.csv

### Result of all locations (API or manually)

We decide to combine all results from API requests, and manually fetch on the web for others the canton and the coordinates. We put all in a CSV file :

In [ ]:
match_location = pd.read_csv('Data/universities_cantons_geo.csv')
match_location.head()

In the dataset we have find 'NPO (Biblioth., Museen, Verwalt.) - NPO', after searching twe find out that si money given all arounf the swiss contry. We have the same result for `Firmen/Privatwirtschaft - FP`.<br/>
For this reason the we decided to remove the Appoved amount of both.<br/>

Furthermore 'Weitere' in english mean other so we can associate the amount of money to a specific canton.
Finnaly for the same reason 'Nicht zuteilbar' mean not assignable, we can't keep it. 


> On le garde ta mega fonction pour sortir le fichier CSV ? :p

## Merge differents data set.

For each university we compute the sum of 'Approved amount'.

In [ ]:
data_swiss_universities = pd.merge(
    snsf_data_swiss[['University', 'Approved Amount']],
    match_location[['University', 'Canton']],
    on=['University'],
    how='inner'
)
data_swiss_universities.head()

In [ ]:
amount_by_university = data_swiss_universities.groupby('University').sum()
amount_by_university.head()

For each cantons we compute the sum of 'Approved amount'.

In [ ]:
amount_by_cantons = data_swiss_universities.groupby('Canton').sum()
amount_by_cantons.head()

In [ ]:
amount_by_cantons.describe()

As we can notice the difference between the max and the min is huge. <br/> the vizualisation is not going to be very representative (with few colors variations) if we take those values. <br/>


> For the reason we decide to apply the fourth root on values in order to reduce the gap between the amount of money given to each cantons.

In [ ]:
amount_by_cantons['Approved Amount Reduced'] = pow(amount_by_cantons['Approved Amount'], 1/5)
amount_by_cantons.head()

In [ ]:
cantons_data = pd.DataFrame({
        'Canton': cantons_id
    })
cantons_data.set_index('Canton', inplace=True)

We need to get the value for each canton, as we can notice in our data some canton does not appear.
> We need to add them with a Appoved Amount equals to 0.

In [ ]:
amount_by_all_cantons = pd.merge(cantons_data, amount_by_cantons, right_index=True, left_index=True, how='left')
amount_by_all_cantons.fillna(0, inplace=True)
amount_by_all_cantons.reset_index(inplace=True)
amount_by_all_cantons.head()

## Vizualisation

In [ ]:
# We build the map from the data got before.
map_final = folium.Map(location=[46.8, 8.1], zoom_start=8)

# You need to have the locale 'fr_CH.utf8' or 'fr_CH.UTF-8' install
try:
    locale.setlocale(locale.LC_MONETARY, 'fr_CH.utf8') # Linux locale
except:
    try:
       locale.setlocale(locale.LC_MONETARY, 'fr_CH.UTF-8') # Mac locale
    except:
        print('Unable to set fr_CH.utf8 or fr_CH.UTF-8 locale, the currency will not be correct')

# Add marker of universities
universities_locations = match_location.set_index('University')[['Latitude', 'Longitude']].dropna()
for university, row in universities_locations.iterrows():
    amount = amount_by_university.loc[university]['Approved Amount']
    message = university + ' (' + locale.currency( amount, grouping=True ) + ')'
    marker = folium.Marker(row[['Latitude', 'Longitude']], popup=message)
    map_final.add_children(marker)
    
value_column = 'Approved Amount Reduced'

scale = list(
    np.linspace(
        amount_by_all_cantons[value_column].min(),
        amount_by_all_cantons[value_column].max(),
        6
    )
)

map_final.choropleth(
    geo_path=topojson_cantons,
    data=amount_by_all_cantons,
    columns=['Canton', value_column],
    key_on='feature.id',
    fill_color='PuBu',
    fill_opacity=0.8,
    line_opacity=0.2,
    threshold_scale=scale,
    legend_name='the fourth root of grant money by Swiss universities canton',
    topojson='objects.cantons'
)

map_final

> Legend

>> Colors: the fourth root of grant money by Swiss universities canton

>> Markers: university names with the grant money